In [3]:
library(data.table)
library(tictoc)


Attaching package: ‘tictoc’


The following object is masked from ‘package:data.table’:

    shift




In [4]:
counts_dt <- fread("bulk_counts_long.csv")
metadata_dt <- fread("sample_metadata.csv")


In [6]:
#avvio timer generale
#versione con task2 senza join: setindex() in data.table non influisce sui join ([x[y, on=...]) — influisce solo sui subset e filtri ([condition]).
#I join in data.table usano:
#le chiavi primarie definite da setkey() (se esistono), oppure l’argomento on= (che internamente fa una ricerca sequenziale o binaria), ma non usano indici secondari.
#Quindi, rifare un join prima e dopo setindex() non cambia nulla in termini di prestazioni.
#Il join continuerà a usare la chiave di sample_metadata (quella impostata con setkey(sample_metadata, sample_id)), e non l’indice secondario della tabella dei conteggi.
#DATA.TABLE
tic("esercizio 3")

tic("Task1")
setkey(metadata_dt, sample_id)
joined_dt <- metadata_dt[counts_dt] #equi.join (tra due set di dati che hanno in comune la colonna sample_id, la chiave) la chiave dovrebbe velocizzare il join
toc() 

print("Risultato Task1") 
print(head(joined_dt), 5) 

#task2
tic("task2")
#definisco parametri su cui fare la query
target_gene <- "GENE_0356"
target_sample <- "S03"

#query
print("Benchmarking SENZA Indice") #valutare prestazioni
tic("Query senza indice")
subset_no_index <- counts_dt[gene == target_gene & sample_id == target_sample]
tempo <- toc(quiet=TRUE) 
t_finale <- tempo$toc - tempo$tic
print(subset_no_index)

#Creazione Indice Secondario su (gene, sample_id)
setindex(counts_dt, gene, sample_id)

print("Benchmarking CON Indice")
tic("Query con indice (ricerca binaria)")
subset_with_index <- counts_dt[gene == target_gene & sample_id == target_sample]
tempo <- toc(quiet=TRUE) 
t_finale2 <- tempo$toc - tempo$tic
print(subset_with_index)

print(paste0("tempo  pre secondary index: ", t_finale, " secondi"))
print(paste0("tempo post secondary index: ", t_finale2, " secondi")) #ci mette meno con index

#i tempi con index sono maggiori per problemi relativi al rumore, andrebbe eseguito il calcolo in loop migliaia di volte
toc()

Task1: 0.004 sec elapsed
[1] "Risultato Task1"
   sample_id condition  batch patient_id timepoint i.sample_id count
      <char>    <char> <char>     <char>    <char>      <char> <int>
1: GENE_0356      <NA>   <NA>       <NA>      <NA>         S01     2
2: GENE_0233      <NA>   <NA>       <NA>      <NA>         S01    13
3: GENE_0232      <NA>   <NA>       <NA>      <NA>         S01    73
4: GENE_0160      <NA>   <NA>       <NA>      <NA>         S01    25
5: GENE_0049      <NA>   <NA>       <NA>      <NA>         S01     9
6: GENE_0245      <NA>   <NA>       <NA>      <NA>         S01    20
[1] "Benchmarking SENZA Indice"
        gene sample_id count
      <char>    <char> <int>
1: GENE_0356       S03     3
[1] "Benchmarking CON Indice"
        gene sample_id count
      <char>    <char> <int>
1: GENE_0356       S03     3
[1] "tempo  pre secondary index: 0.00300000000015643 secondi"
[1] "tempo post secondary index: 0.00199999999995271 secondi"
task2: 0.025 sec elapsed


In [6]:
library(tictoc)
#versione con data.frame
counts_df <- read.csv("bulk_counts_long.csv", stringsAsFactors = FALSE)
metadata_df <- read.csv("sample_metadata.csv", stringsAsFactors = FALSE)

#avvio timer generale
tic("esercizio 3, data.frame")

# TASK 1
joined_df <- merge(metadata_df, counts_df, by = "sample_id")
toc()

print("Risultato Task1")
print(head(joined_df, 5))


# TASK 2
tic("task2")

# Definisci i parametri su cui fare la query
target_gene <- "GENE_0356"
target_sample <- "S03"

# Benchmarking SENZA Indicizzazione Ottimizzata
print("Benchmarking SENZA Indice Ottimizzato")
tic("Query senza indice")
# Si usa la sintassi di subsetting standard di data.frame (ricerca lineare)
subset_no_index_df <- joined_df[joined_df$gene == target_gene & joined_df$sample_id == target_sample, ]
tempo <- toc(quiet=TRUE)
t_finale <- tempo$toc - tempo$tic
print(subset_no_index_df)

joined_post_df <- joined_df[order(joined_df$gene, joined_df$sample_id), ]

print("Benchmarking CON Riordinamento (Simulazione Indice)")
tic("Query con riordinamento (DF)")
subset_with_index_df <- joined_post_df[joined_post_df$gene == target_gene & joined_post_df$sample_id == target_sample, ]
tempo <- toc(quiet=TRUE)
t_finale2 <- tempo$toc - tempo$tic
print(subset_with_index_df)

print(paste0("tempo pre  secondary index (DF): ", t_finale, " secondi"))
print(paste0("tempo post secondary index (DF): ", t_finale2, " secondi")) 

# stop il timer generale
toc()

esercizio 3, data.frame: 0.017 sec elapsed
[1] "Risultato Task1"
  sample_id condition  batch patient_id timepoint      gene count
1       S01   treated batch1       P014        T0 GENE_0356     2
2       S01   treated batch1       P014        T0 GENE_0233    13
3       S01   treated batch1       P014        T0 GENE_0232    73
4       S01   treated batch1       P014        T0 GENE_0160    25
5       S01   treated batch1       P014        T0 GENE_0049     9
[1] "Benchmarking SENZA Indice Ottimizzato"
     sample_id condition  batch patient_id timepoint      gene count
1001       S03   treated batch2       P017        T1 GENE_0356     3
[1] "Benchmarking CON Riordinamento (Simulazione Indice)"
     sample_id condition  batch patient_id timepoint      gene count
1001       S03   treated batch2       P017        T1 GENE_0356     3
[1] "tempo pre  secondary index (DF): 0.0029999999999859 secondi"
[1] "tempo post secondary index (DF): 0.00200000000000955 secondi"
task2: 0.039 sec elapsed


In [8]:
#tabella confronto tempi
T_DT1 <- 0.018
T_DT_pre <- 0.010
T_DT_post <- 0.005
T_DT2 <- 0.025
T_DF1 <- 0.017
T_DF_pre <- 0.003
T_DF_post <- 0.002
T_DF2 <- 0.039


# Creazione della tabella riassuntiva
risultati_performance <- data.frame(
  Task = c("Task 1",
           "pre index",
           "post index",
           "Task 2"),
  
  Tempo_data.table_Sec = c(T_DT1, T_DT_pre, T_DT_post, T_DT2),
  Tempo_data.frame_Sec = c(T_DF1, T_DF_pre, T_DF_post, T_DF2)
)

# Aggiungiamo una colonna per il fattore di velocizzazione (Speedup)
risultati_performance$Speedup_DT_vs_DF <- 
  round(risultati_performance$Tempo_data.frame_Sec / risultati_performance$Tempo_data.table_Sec, 1)

# Stampiamo la tabella finale
print(risultati_performance)

        Task Tempo_data.table_Sec Tempo_data.frame_Sec Speedup_DT_vs_DF
1     Task 1                0.018                0.017              0.9
2  pre index                0.010                0.003              0.3
3 post index                0.005                0.002              0.4
4     Task 2                0.025                0.039              1.6
